In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import pickle
from sklearn.utils.class_weight import compute_class_weight
import statistics

In [ ]:
device = torch.device('cuda')

In [ ]:
rng = np.random.default_rng(73512)

In [ ]:
with open(r'train.pkl', 'rb') as f:
    data = pickle.load(f)

with open(r'test.pkl', 'rb') as f:
    verify_data = pickle.load(f)

# Data preprocessing

In [ ]:
from sklearn.preprocessing import OneHotEncoder

categories = [row[1] for row in data]

class_weights = torch.tensor(compute_class_weight('balanced', classes=[0,1,2,3,4], y=categories)).to(device)
print(class_weights)

encoder = OneHotEncoder()
categories_2d = [[category] for category in categories]
onehot_encoded = encoder.fit_transform(categories_2d)
onehot_encoded_array = onehot_encoded.toarray()

data_modified = []
targets_modified = []
for i in range(len(data)):
    row = list(data[i][0])
    data_modified.append(row)
    row2 = onehot_encoded_array[i]
    targets_modified.append(row2)
data = [torch.tensor(row).to(torch.float32) for row in data_modified]
targets = [torch.tensor(row).to(torch.float32) for row in targets_modified]

In [ ]:
from torch.utils.data import Dataset

class VariableLenDataset(Dataset):
    def __init__(self, in_data, target):
        self.data = [(x, y) for x, y in zip(in_data, target)]      

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        in_data, target = self.data[idx]
        return in_data, target

In [ ]:
data = np.array(data)
targets = np.array(targets)
train_indices = rng.random(len(data)) > 0.3
test_indices = ~train_indices
train_indices = np.where(train_indices==True)[0]
test_indices = np.where(test_indices==True)[0]
train_set = VariableLenDataset(data[train_indices], targets[train_indices])
test_set = VariableLenDataset(data[test_indices], targets[test_indices])

In [ ]:
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

pad = 0

def pad_collate(batch, pad_value=0):
    xx, yy = zip(*batch)
    x_lens = [len(x) for x in xx]
    y_lens = [len(y) for y in yy]

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=pad_value)
    yy_pad = pad_sequence(yy, batch_first=True, padding_value=pad_value)

    return xx_pad, yy_pad, x_lens, y_lens

# LSTM model

In [ ]:
class LSTMRegressor(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, out_size):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers)
        self.fc = nn.Linear(hidden_size, out_size)
        
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        state = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return hidden, state
    
    def forward(self, x, hidden):
        x = torch.transpose(x, 0, 1)
        all_outputs, hidden = self.lstm(x, hidden)
        out = all_outputs[-1]
        x = self.fc(out)
        return x, hidden

# Parameters

In [ ]:
train_loader = DataLoader(train_set, batch_size=10, shuffle=True, collate_fn=pad_collate)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, drop_last=False, collate_fn=pad_collate)
model = LSTMRegressor(1, 200, 3, 5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fun = nn.CrossEntropyLoss(weight=class_weights,reduction='mean')

# Training

In [ ]:
for epoch in range(1):
    for x, targets, x_len, target_len in train_loader:
        x = x.to(device).unsqueeze(2)
        targets = targets.to(device)
        hidden, state = model.init_hidden(x.size(0))
        hidden, state = hidden.to(device), state.to(device) 
        
        preds, _ = model(x, (hidden, state))        
        preds = preds.squeeze(1)
        optimizer.zero_grad()
        loss = loss_fun(preds, targets)
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, loss: {loss.item():.3}")

In [ ]:
with torch.no_grad():
    true_preds = 0
    num_preds = 0
    true_classes_preds = np.array([0,0,0,0,0])
    num_classes_preds = np.array([0,0,0,0,0])
    for x, targets, x_len, target_len in test_loader:
        x = x.to(device).unsqueeze(2)
        targets = targets.to(device)
        hidden, state = model.init_hidden(x.size(0))
        hidden, state = hidden.to(device), state.to(device) 
        preds, _ = model(x, (hidden, state))
        preds = preds.squeeze(1)
        preds = (torch.round(preds).int()).long()
        if (torch.argmax(preds) == torch.argmax(targets)):
            true_preds +=1
            true_classes_preds[torch.argmax( 
                targets).item()]+=1
        num_classes_preds[torch.argmax(targets).item()]+=1
        num_preds += targets.shape[0]
acc = true_preds/num_preds
classes_acc = true_classes_preds/num_classes_preds
print("Number of classes predictions:",true_classes_preds)
print("Number of classes labels",num_classes_preds)
print("Classes accuracy:",classes_acc)
print("Mean:",statistics.mean(classes_acc))
print("Euclidean norm:",np.linalg.norm(np.array(classes_acc))/5)
print("Accuracy:",acc)


# Verify data with no labels

In [ ]:
from sklearn.preprocessing import OneHotEncoder

verify_categories = [row[1] for row in verify_data]


verify_data_modified = []
for i in range(len(verify_data)):
    row = list(verify_data[i])
    verify_data_modified.append(row)
verify_data = [torch.tensor(row).to(torch.float32) for row in verify_data_modified]

In [ ]:
from torch.utils.data import Dataset

class VerifyVariableLenDataset(Dataset):
    def __init__(self, in_data):
        self.data = in_data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        in_data = self.data[idx]
        return in_data

In [ ]:
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

pad = 0

def verify_pad_collate(batch, pad_value=0):
    xx = batch
    x_lens = [len(x) for x in xx]

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=pad_value)
    
    return xx_pad, x_lens

In [ ]:
verify_data = np.array(verify_data)
verify_set = VerifyVariableLenDataset(verify_data)
verify_loader = DataLoader(verify_set, batch_size=1, shuffle=False, collate_fn=verify_pad_collate)

In [ ]:
import csv
predictions = []
with torch.no_grad():
    num_classes_preds = np.array([0,0,0,0,0])
    for x, x_len in verify_loader:
        x = x.to(device).unsqueeze(2)
        hidden, state = model.init_hidden(x.size(0))
        hidden, state = hidden.to(device), state.to(device) 
        preds, _ = model(x, (hidden, state))
        preds = preds.squeeze(1)
        preds = (torch.round(preds).int()).long()
        preds = torch.argmax(preds).item()
        num_classes_preds[preds]+=1
        predictions.append(preds)
    print("Classes predictions: ",num_classes_preds)


In [ ]:
with open("predictions.csv", 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(predictions)
